-----
-----
-----

#### **running_mean ve running_var, tek bir batch’in değil; eğitim boyunca görülen tüm batch’lerin yumuşatılmış istatistikleridir. Anlık değil, birikimli ve inference için tasarlanmıştır.**

>## “running_mean / running_var TAM OLARAK NEYİN istatistiği?”

**Bunlar, BatchNorm katmanına GİREN feature map’lerin, KANAL BAZINDA dağılım istatistikleridir.**

Model diyor ki:

* “Ben bugüne kadar binlerce görüntü gördüm.
* Bu katmandaki feature’lar genelde böyle dağılıyor.”

İşte bu “genelde böyle” dediği şey:

* running_mean

* running_var,


### 🧠 Mantığı çok sade anlatayım :: Her iterasyonda BN şunu yapar (train modunda)
```python
batch_mean = mean(x_batch)
batch_var  = var(x_batch)

running_mean = (1 - momentum) * running_mean + momentum * batch_mean
running_var  = (1 - momentum) * running_var  + momentum * batch_var
```
📌 momentum genelde 0.1

**Momentum = geçmişe ne kadar kulak vereceğiz?** ---> *“Yeni gelen batch mi daha önemli, yoksa geçmişte gördüklerim mi?”*

Yani:

* yeni batch %10 etkiliyor

* eski geçmiş %90 korunuyor

| Kavram         | Ne zaman?            | Kaynak                    |
| -------------- | -------------------- | ------------------------- |
| `batch_mean`   | Train                | O anki mini-batch         |
| `running_mean` | Train’de güncellenir | Geçmiş batch’lerin EMA’sı |
| `running_mean` | Eval/Inference       | **KULLANILIR**            |
| `batch_mean`   | Eval/Inference       | ❌ Kullanılmaz             |


### 📌 "Mean" şu:
**mean = ortalama(X[b,c,h,w])**

➡️ Bu kanalın aktivasyonlarının ortalaması

### 📌 “Var” neyin varyansı?

Aynı kanal için = varyans(X[b,c,h,w])

➡️ Bu kanalın aktivasyonlarının ne kadar yayıldığı

**BatchNorm’daki running_mean ve running_var, her kanalın eğitim boyunca ürettiği feature map aktivasyonlarının ortalama ve yayılım istatistikleridir; input’un değil, modelin iç temsillerinin istatistiğidir.**

-----
-----
-----


# Frozen BatchNorm — Baştan Sona Kod + Satır Satır Mantık

Bu defterin hedefi:
1) **FrozenBatchNorm2d** katmanını **en temelden** inşa etmek  
2) Yazılan **her satırın ne yaptığını** açıklamak  
3) **BN → FrozenBN dönüştürme** fonksiyonunu (recursive) anlamak  
4) En sonda: FrozenBN kullanan **basit bir CNN model** örneği

> Bu defter NCHW/BCHW formatını (PyTorch standardı) kullanır: `(B, C, H, W)`.


---

## 0) Ön bilgi: FrozenBN ne yapar?

**FrozenBN = BatchNorm’un eval (inference) davranışını sürekli kullanmak.**

- `running_mean` ve `running_var` **sabit** (güncellenmez)
- `gamma (weight)` ve `beta (bias)` çoğunlukla **sabit** (öğrenilmez)
- Forward’da yine normalize eder:

\[
y = \frac{x - running\_mean}{\sqrt{running\_var + \epsilon}} \cdot \gamma + \beta
\]

**Norm var, öğrenme yok.**


In [1]:
import torch
import torch.nn as nn

---

## 1) FrozenBatchNorm2d: Sıfırdan Tanımlama (satır satır)

Aşağıdaki sınıf, FrozenBN’in çekirdeğidir.

Önemli parçalar:
- `register_buffer`: **öğrenilmeyen** ama `state_dict` içinde saklanan değerler
- `nn.Parameter`: **öğrenilebilir** parametreler (istersek sonradan freeze edebiliyoruz)
- `forward`: normalize formülü


### 1.1 Kod (yorum satırlarıyla)

Kodun içinde her kritik satırın yanında kısa yorum var.  
Altındaki bölümde ise **satır satır açıklama** var.


In [2]:
class FrozenBatchNorm2d(nn.Module):
    # 1) nn.Module'den kalıtım: PyTorch katmanı olabilmek için gerekli
    def __init__(self, num_features: int, eps: float = 1e-5, affine: bool = True):
        # 2) üst sınıfın init'i: parametre/buffer kayıt mekanizması için şart
        super().__init__()

        # 3) Kanal sayısı (C). BN/FrozenBN kanal başına çalışır
        self.num_features = int(num_features)

        # 4) Sayısal stabilite için küçük sabit: sqrt(var + eps)
        self.eps = float(eps)

        # 5) Affine var mı? (gamma/beta kullanılsın mı)
        self.affine = bool(affine)

        # 6) running_mean: inference'ta kullanılan ortalama (C boyutunda)
        # register_buffer -> gradient almaz, optimizer güncellemez, ama state_dict'e kaydolur
        self.register_buffer("running_mean", torch.zeros(self.num_features))

        # 7) running_var: inference'ta kullanılan varyans (C boyutunda)
        self.register_buffer("running_var", torch.ones(self.num_features))

        # 8) Eğer affine=True ise gamma/beta tanımla:
        # Bunlar normalde BN'de learnable parametrelerdir.
        # Frozen kullanımında çoğunlukla requires_grad=False yapılır.
        if self.affine:
            self.weight = nn.Parameter(torch.ones(self.num_features))  # gamma
            self.bias   = nn.Parameter(torch.zeros(self.num_features)) # beta
        else:
            # affine yoksa parametre kaydetme (None)
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 9) x beklenen şekil: (B, C, H, W)
        # C boyutu self.num_features ile aynı olmalı
        if x.dim() != 4:
            raise ValueError(f"FrozenBatchNorm2d 4D tensor bekler (B,C,H,W). Geldi: {tuple(x.shape)}")

        # 10) running_mean/var'ı broadcast edilebilir şekle getir:
        # (C,) -> (1,C,1,1)
        mean = self.running_mean.view(1, -1, 1, 1)
        var  = self.running_var.view(1, -1, 1, 1)

        # 11) Normalizasyon: (x - mean) / sqrt(var + eps)
        x_hat = (x - mean) / torch.sqrt(var + self.eps)

        # 12) Affine varsa gamma/beta uygula
        if self.affine:
            w = self.weight.view(1, -1, 1, 1)  # (1,C,1,1)
            b = self.bias.view(1, -1, 1, 1)
            return x_hat * w + b

        # 13) Affine yoksa sadece normalize edilmiş çıktı
        return x_hat

### 1.2 Satır satır açıklama

**(1)** `class FrozenBatchNorm2d(nn.Module)`  
- Katmanı PyTorch ekosistemine sokar. `forward()` yazınca otomatik çalışır.

**(2)** `super().__init__()`  
- Parametre ve buffer kayıt sistemi burada başlar.

**(3)** `self.num_features`  
- Kanal sayısı. FrozenBN kanal başına (C) farklı mean/var tutar.

**(4)** `self.eps`  
- `sqrt(var + eps)` ile 0'a bölmeyi engeller.

**(5)** `self.affine`  
- `gamma/beta` uygulanacak mı? Bazı yerlerde kapatılır.

**(6)-(7)** `register_buffer("running_mean/var", ...)`  
- Bunlar “öğrenilen parametre” değil, “sabit istatistik”.  
- Optimizer bunları **güncellemez**.  
- Ama `state_dict` içinde kaydedilir/yüklenir.

**(8)** `weight/bias`  
- BN’deki `gamma/beta`. FrozenBN’de genelde **freeze** edilir ama forward’da kullanılabilir.

**(9)** shape kontrol  
- FrozenBatchNorm2d, `Conv2d` çıktısı gibi 4D bekler.

**(10)** `view(1, -1, 1, 1)`  
- (C,) vektörünü, feature map'e broadcast etmek için.

**(11)** normalize formülü  
- Burada norm gerçekten yapılıyor. “eval mod” demek “norm yok” demek değildir.

**(12)** affine  
- Normalize edilmiş değerleri yeniden ölçekler (`gamma`) ve kaydırır (`beta`).

**(13)** affine kapalıysa  
- Saf normalize çıktı döner.


---

## 2) FrozenBN'i gerçekten "freeze" etmek

Yukarıdaki sınıf, gamma/beta'yı `Parameter` olarak tutuyor.
Frozen kullanımında çoğu pratik:

- `weight.requires_grad = False`
- `bias.requires_grad = False`

Böylece optimizer bunları güncellemez.

Aşağıda küçük yardımcı fonksiyon:


In [3]:
def freeze_frozenbn_params(module: nn.Module):
    """Model içindeki FrozenBatchNorm2d katmanlarının weight/bias'ını dondur."""
    for m in module.modules():
        if isinstance(m, FrozenBatchNorm2d):
            if m.weight is not None:
                m.weight.requires_grad_(False)
            if m.bias is not None:
                m.bias.requires_grad_(False)

---

## 3) BN → FrozenBN dönüştürme (en pratik yöntem)

Gerçek hayatta FrozenBN genelde şu şekilde kullanılır:
- Pretrained model (ResNet vs.) içinde **BatchNorm2d** vardır
- Fine-tune yaparken BN kararsızlık yaratmasın diye
  - BN katmanlarını FrozenBN ile değiştirirsin
  - BN’nin running_mean/var + weight/bias değerlerini FrozenBN’e kopyalarsın

Aşağıdaki fonksiyon bunu **recursive** (ağaç gibi tüm alt modüllere) yapar.


### 3.1 Kod (yorumlu)


In [4]:
def convert_bn_to_frozenbn(module: nn.Module) -> nn.Module:
    """Model içindeki nn.BatchNorm2d'leri FrozenBatchNorm2d ile değiştirir (recursive)."""

    # 1) module.named_children(): sadece bir seviye alt modülleri verir
    # list(...) kullanıyoruz çünkü iterasyon sırasında set edeceğiz
    for name, child in list(module.named_children()):

        # 2) Önce çocuğun altını da dönüştür (recursive)
        new_child = convert_bn_to_frozenbn(child)

        # 3) Eğer child bir BatchNorm2d ise, FrozenBN ile değiştir
        if isinstance(new_child, nn.BatchNorm2d):
            bn = new_child

            # 4) FrozenBN'i aynı kanal sayısıyla oluştur
            fb = FrozenBatchNorm2d(
                num_features=bn.num_features,
                eps=bn.eps,
                affine=bn.affine
            )

            # 5) BN'den FrozenBN'e parametre/buffer kopyala
            with torch.no_grad():
                # running stats
                fb.running_mean.copy_(bn.running_mean)
                fb.running_var.copy_(bn.running_var)

                # affine varsa gamma/beta
                if bn.affine:
                    fb.weight.copy_(bn.weight)
                    fb.bias.copy_(bn.bias)

            # 6) Frozen kullanım: gamma/beta öğrenilmesin
            freeze_frozenbn_params(fb)

            # 7) Parent module içinde bu child'ı değiştir
            setattr(module, name, fb)
        else:
            # 8) BN değilse, dönüştürülmüş alt modülü yerine koy
            setattr(module, name, new_child)

    # 9) En sonda güncellenmiş modülü döndür
    return module


### 3.2 Satır satır açıklama

**(1)** `named_children()`  
- Sadece doğrudan alt modüller. (Sequential içindeki bloklar vs.)

**(2)** recursion  
- Model ağacı derindir: blokların içinde bloklar olabilir. O yüzden önce aşağı iniyoruz.

**(3)** `isinstance(..., nn.BatchNorm2d)`  
- BN2d gördüğümüz yeri hedefliyoruz.

**(4)** FrozenBN oluşturma  
- BN ile aynı `num_features` ve `eps` ile kuruyoruz.

**(5)** Kopyalama  
- En kritik adım: pretrained running_mean/var ve gamma/beta FrozenBN’e taşınır.  
- Böylece FrozenBN, BN’nin eval davranışını taklit eder.

**(6)** Freeze  
- `requires_grad=False` ile öğrenmeyi kapatıyoruz.

**(7)-(8)** `setattr`  
- Parent modülün ilgili attribute’una yeni katmanı yazıyoruz.

**(9)** dönüş  
- Recursion zincirinin üstüne geri döner.


---

## 4) BN eval vs FrozenBN: Hızlı kontrol

Aynı input için:
- BN `eval()` modunda
- FrozenBN forward

çıktılar yakın olmalıdır (küçük sayısal farklar olabilir).


In [5]:
torch.manual_seed(0)

# BN'li mini blok
bn_block = nn.Sequential(
    nn.Conv2d(3, 8, 3, padding=1, bias=False),
    nn.BatchNorm2d(8),
    nn.ReLU(inplace=True),
)

# running stats oluşsun diye train forward
bn_block.train()
for _ in range(10):
    _ = bn_block(torch.randn(4, 3, 32, 32))

# Frozen'a çevir
frozen_block = convert_bn_to_frozenbn(bn_block)

x = torch.randn(2, 3, 32, 32)

bn_block.eval()
with torch.no_grad():
    y_bn = bn_block(x)
    y_fr = frozen_block(x)

print("Mean abs diff:", (y_bn - y_fr).abs().mean().item())

Mean abs diff: 0.0


---

## 5) FrozenBN kullanan basit model 

Aşağıdaki model:
- Basit CNN backbone
- İçinde FrozenBatchNorm2d kullanır
- İncelemek için “düz” tutuldu

Bu modelin amacı:
- FrozenBN’in **gerçek bir ağ içinde nerede durduğunu** göstermek


In [6]:
class ConvFrozenBNAct(nn.Module):
    """Conv2d -> FrozenBN -> Act"""
    def __init__(self, in_ch, out_ch, k=3, s=1, p=None, act='silu'):
        super().__init__()
        if p is None:
            p = k // 2
        self.conv = nn.Conv2d(in_ch, out_ch, k, stride=s, padding=p, bias=False)
        self.bn = FrozenBatchNorm2d(out_ch, eps=1e-5, affine=True)
        freeze_frozenbn_params(self.bn)  # pratikte frozen

        if act == 'relu':
            self.act = nn.ReLU(inplace=True)
        elif act == 'silu':
            self.act = nn.SiLU(inplace=True)
        else:
            self.act = nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))


class SimpleFrozenBN_CNN(nn.Module):
    """FrozenBN kullanan basit CNN"""
    def __init__(self, num_classes=10, base=32):
        super().__init__()
        self.stem = nn.Sequential(
            ConvFrozenBNAct(3, base, k=3, s=2, act='silu'),
            ConvFrozenBNAct(base, base, k=3, s=1, act='silu'),
        )
        self.stage1 = nn.Sequential(
            ConvFrozenBNAct(base, base*2, k=3, s=2, act='silu'),
            ConvFrozenBNAct(base*2, base*2, k=3, s=1, act='silu'),
        )
        self.stage2 = nn.Sequential(
            ConvFrozenBNAct(base*2, base*4, k=3, s=2, act='silu'),
            ConvFrozenBNAct(base*4, base*4, k=3, s=1, act='silu'),
        )
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(base*4, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)


### 5.1 Bu modelde FrozenBN “hangi mantıkla” var?

- Conv, feature map üretir: `(B,C,H,W)`
- FrozenBN, her kanalı:
  - `running_mean[c]` ve `running_var[c]` ile normalize eder
  - `gamma/beta` ile scale/shift yapar (ama parametreler frozen)
- Aktivasyon (SiLU/ReLU) sonrası bir sonraki katmana geçer

Bu tasarım:
- Detection/segmentation fine-tune senaryosunda backbone’ta sık görülür
- Çünkü küçük batch ile BN istatistikleri bozulmasın diye “kilitlenir”


In [7]:
# Hızlı çalıştırma testi
if __name__ == "__main__":
    torch.manual_seed(1)
    model = SimpleFrozenBN_CNN(num_classes=10, base=32)
    x = torch.randn(2, 3, 224, 224)
    y = model(x)
    print("Output shape:", y.shape)


Output shape: torch.Size([2, 10])


---

## 6) Kapanış: En kısa özet

- FrozenBN, BN’nin **inference formülünü** sürekli uygular.
- running_mean/var **sabit** tutulur.
- gamma/beta çoğunlukla **freeze** edilir.
- Dönüştürme fonksiyonu, BN değerlerini FrozenBN’e **kopyalar** ve modeli günceller.
